# Data Types: Maps

In [1]:
import riak

client = riak.RiakClient()

In [2]:
client.is_alive()

True

http://docs.basho.com/riak/kv/2.2.3/developing/data-types/maps/

## Create a map

For this example, say we want to use Riak KV to store information about our company’s customers. We’ll use the maps bucket type created and activated previously and a bucket called customers. Each customer’s data will be contained in its own key in the customers bucket.

We can create a map for the user Ahmed (ahmed_info) using the maps bucket type:

In [5]:
customers = client.bucket_type('maps').bucket('customers')
customer_map = customers.new('ahmed_info')

## Registers

Registers are essentially named binaries (like strings). Any binary value can act as the value of a register. Like flags, registers cannot be used on their own and must be embedded in Riak maps.

## Register with maps

In [6]:
customer_map.registers['first_name'].assign('Ahmed')
customer_map.registers['phone_number'].assign('5551234567')

# Integers need to be stored as strings and then converted back when the
# data is retrieved. The following would work as well:
customer_map.registers['phone_number'].assign(str(5551234567))

customer_map.store()

If a register did not previously exist, Riak KV will create that register for you.

## Flags

Flags behave much like Boolean values, except that instead of true or false flags have the values enable or disable.

Flags cannot be used on their own, i.e. a flag cannot be stored in a bucket/key by itself. Instead, flags can only be stored within maps.

To disable an existing flag, you have to read it or provide a context.

## Flags within maps

Now let’s say that we add an Enterprise plan to our pricing model. We’ll create an enterprise_customer flag to track whether Ahmed has signed up for the new plan. He hasn’t yet, so we’ll set it to false:

In [8]:
customer_map.flags['enterprise_customer'].disable()
customer_map.store()

We can retrieve the value of that flag at any time:

In [9]:
customer_map.reload().flags['enterprise_customer'].value

False

## Counters within maps

We also want to know how many times Ahmed has visited our website. We’ll use a page_visits counter for that and run the following operation when Ahmed visits our page for the first time:

In [10]:
customer_map.counters['page_visits'].increment()
customer_map.store()

Even though the page_visits counter did not exist previously, the above operation will create it (with a default starting point of 0) and the increment operation will bump the counter up to 1.

## Sets within maps

We’d also like to know what Ahmed’s interests are so that we can better design a user experience for him. Through his purchasing decisions, we find out that Ahmed likes robots, opera, and motorcycles. We’ll store that information in a set inside of our map:

In [11]:
for interest in ['robots', 'opera', 'motorcycles']:
    customer_map.sets['interests'].add(interest)
customer_map.store()

We can then verify that the interests set includes these three interests:

In [12]:
reloaded_map = customer_map.reload()
for interest in ['robots', 'opera', 'motorcycles']:
    print(interest in reloaded_map.sets['interests'].value)

True
True
True


We learn from a recent purchasing decision that Ahmed actually doesn’t seem to like opera. He’s much more keen on indie pop. Let’s change the interests set to reflect that:

In [13]:
customer_map.sets['interests'].discard('opera')
customer_map.sets['interests'].add('indie pop')
customer_map.store()

## Maps within maps

We’ve stored a wide of variety of information—of a wide variety of types—within the ahmed_info map thus far, but we have yet to explore recursively storing maps within maps (which can be nested as deeply as you wish).

Our company is doing well and we have lots of useful information about Ahmed, but now we want to store information about Ahmed’s contacts as well. We’ll start with storing some information about Ahmed’s colleague Annika inside of a map called annika_info.

First, we’ll store Annika’s first name, last name, and phone number in registers:

In [14]:
customer_map.maps['annika_info'].registers['first_name'].assign('Annika')
customer_map.maps['annika_info'].registers['last_name'].assign('Weiss')
customer_map.maps['annika_info'].registers['phone_number'].assign(str(5559876543))
customer_map.store()

The value of a register in a map can be obtained without a special method:

In [15]:
customer_map.reload().maps['annika_info'].registers['first_name'].value

'Annika'

Registers can also be removed:

In [16]:
del customer_map.maps['annika_info'].registers['first_name']
customer_map.store()

Now, we’ll store whether Annika is subscribed to a variety of plans within the company as well:

In [17]:
customer_map.maps['annika_info'].flags['enterprise_plan'].disable()
customer_map.maps['annika_info'].flags['family_plan'].disable()
customer_map.maps['annika_info'].flags['free_plan'].enable()
customer_map.store()

The value of a flag can be retrieved at any time:

In [15]:
customer_map.reload().maps['annika_info'].flags['enterprise_plan'].value

False

It’s also important to track the number of purchases that Annika has made with our company. Annika just made her first widget purchase:

In [18]:
customer_map.maps['annika_info'].counters['widget_purchases'].increment()
customer_map.store()

Now let’s store Annika’s interests in a set:

In [19]:
customer_map.maps['annika_info'].sets['interests'].add('tango dancing')
customer_map.store()

We can remove that interest in just the way that we would expect:

In [20]:
customer_map.maps['annika_info'].sets['interests'].discard('tango dancing')
customer_map.store()

If we wanted to add store information about one of Annika’s specific purchases, we could do so within a map:

In [19]:
customer_map.maps['annika_info'].maps['purchase'].flags['first_purchase'].enable()
customer_map.maps['annika_info'].maps['purchase'].registers['amount'].assign(str(1271))
customer_map.maps['annika_info'].maps['purchase'].sets['items'].add('large widget')
# and so on
customer_map.store()

In [21]:
customer_map.reload().maps['annika_info'].maps['purchase'].flags['first_purchase'].value

False